In [36]:
import pandas as pd
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import SGDClassifier


df = pd.read_excel('Immunotherapy.xlsx')

x = df[['sex','age','Time','Number_of_Warts', 'Type','Area','induration_diameter']]
y = df[['Result_of_Treatment']]

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25, random_state=0)

clf = QuadraticDiscriminantAnalysis()
clf.fit(x_treino, y_treino.values.ravel())

print(y_treino)
print(y_teste)
print(clf.score(x_teste, y_teste))

    Result_of_Treatment
48                    1
80                    1
54                    1
73                    1
79                    0
82                    0
3                     1
62                    1
75                    1
27                    0
18                    1
50                    1
51                    1
74                    1
59                    1
55                    1
4                     1
15                    1
17                    1
40                    1
38                    1
5                     1
81                    1
68                    1
0                     1
34                    1
28                    0
11                    1
35                    0
23                    1
..                  ...
32                    1
76                    1
14                    1
85                    1
19                    1
29                    1
49                    0
87                    0
88                    0
69              